### Load packages

In [1]:
from pathlib import Path
import pandas as pd
import json
from datetime import datetime
import logging
from functools import reduce
#import aggregate_funcs.bpm_features as bpm_features
import numpy as np

In [2]:
import pydata_google_auth as pgauth
from google.cloud import bigquery
creds = pgauth.get_user_credentials(['https://www.googleapis.com/auth/cloud-platform'])

In [3]:
import sys
import os
#repo_dir = Path('__file__').parent.parent.absolute()
sys.path.append('/root/fitbit_biomarker_automation/.')
import aggregate_funcs.bpm_features as bpm_features

### Utility function

In [4]:
def preprocess(df):
    '''
    Preprocessing function remove duplicates
    return "datetime_local" as the index of the dataframe
    '''
    df=df.drop_duplicates()
    return df.set_index(pd.DatetimeIndex(df['datetime_local'])).sort_index()

In [18]:
def rename_cols(df,newcols):
    '''
    Rename the columns of a dataframe
    df: pandas dataframe
    newcols = dictionary containing old column names and
              new column names
    '''
    return df.rename(columns = newcols)

def aggregate_table(df,col,group_col,metrics):
    '''
    Aggregate the values of the dataframe column based from
        the defined metrics
    df: pandas dataframe
    col: column name that will be aggregated
    metrics: list of aggregation methods
    '''
    df =  df[col].groupby(group_col).agg(metrics)
    #flatten multilevel index from aggregation
    if isinstance(df.keys(), pd.core.indexes.multi.MultiIndex):
        df.columns = df.columns.droplevel()
    return df

In [7]:
def join_tables(df_list):
    '''
    Join multiple tables by `participant_id`
    df_list = list of pandas dataframe with "participant_id" col
    '''
    df_temp = reduce(
        lambda  left,right: pd.merge(
            left,right,on=['pim_id'],
            how='outer'
        ), 
        df_list)
        
    return df_temp

### Load test dataset

In [8]:
sql_script="""
WITH
  bpm_stream AS(
  SELECT
    pim_id,
    heart_rate_datetime_loc AS datetime_local,
    bpm
  FROM
    `research-01-217611.odp_level2.fitbit_heart_rate`
  WHERE
    DATE(heart_rate_datetime_loc) > DATE_SUB('2021-05-04', INTERVAL 2 DAY)
    AND DATE(heart_rate_datetime_loc) <= '2021-05-04'
    AND bpm >= 40 ),
  sleep_stages AS(
  SELECT
    DISTINCT pim_id,
    sleep_datetime_loc AS sleep_stage_start_loc,
    CAST(TIMESTAMP_ADD( CAST(sleep_datetime_loc AS TIMESTAMP), INTERVAL (sleep_duration -1) SECOND) AS DATETIME) AS sleep_stage_end_loc,
    sleep_stage
  FROM
    odp_level2.fitbit_sleep_detail ),
  sleep_start_end AS (
  SELECT
    pim_id,
    sleep_date_loc,
    start_time_loc AS sleep_start_time,
    CAST(TIMESTAMP_ADD( CAST(start_time_loc AS TIMESTAMP), INTERVAL 10 MINUTE )AS DATETIME) AS sleep_start_time_10mins,
    end_time_loc AS sleep_end_time,
    CAST(TIMESTAMP_ADD( CAST(end_time_loc AS TIMESTAMP), INTERVAL 10 MINUTE )AS DATETIME) AS sleep_end_time_10mins,
  FROM
    `research-01-217611.odp_level2.fitbit_sleeps`
  WHERE
    /* 
	today's complete sleep data may be available tomorrow
	Example, a person sleeps from 9PM (today) to 5AM tomorrow (local time),
	his/her data will possibly not be complete yet today's data upload, 
	so we summarize his/her sleep cycles from yesterday's sleep details
	(Example: {{ ds }} = '2021-04-27'
	   sleep_date_loc: '2021-04-26'
	   start_time_loc: '2021-04-26T23:00:00
	   end_time_loc:   '2021-04-27T04:00:00
	*/ sleep_date_loc = DATE_SUB('2021-05-04', INTERVAL 1 DAY)),
  combine_sleep_info AS (
  SELECT
    a.pim_id,
    a.sleep_date_loc,
    a.sleep_start_time,
    a.sleep_end_time,
    b.sleep_stage_start_loc,
    b.sleep_stage_end_loc,
    b.sleep_stage
  FROM
    sleep_start_end a
  LEFT JOIN
    sleep_stages b
  ON
    a.pim_id = b.pim_id
  WHERE
    (b.sleep_stage IS NOT NULL)
    AND (b.sleep_stage_start_loc BETWEEN a.sleep_start_time
      AND a.sleep_end_time)) (
  SELECT
    DISTINCT PARSE_DATE("%F",
      '2021-05-04') AS ds,
    a.pim_id,
    a.datetime_local,
    a.bpm,
    b.sleep_stage_start_loc AS start_time,
    b.sleep_stage_end_loc AS end_time,
    DATETIME_DIFF(b.sleep_stage_end_loc,
      b.sleep_stage_start_loc,
      MINUTE) AS duration,
    LOWER(b.sleep_stage) AS sleep_event,
  FROM
    bpm_stream a
  LEFT JOIN
    combine_sleep_info b
  ON
    a.pim_id = b.pim_id
  WHERE
    a.datetime_local BETWEEN b.sleep_stage_start_loc
    AND b.sleep_stage_end_loc)
UNION ALL (
  SELECT
    DISTINCT PARSE_DATE("%F",
      '2021-05-04') AS ds,
    a.pim_id,
    a.datetime_local,
    a.bpm,
    b.sleep_start_time AS start_time,
    b.sleep_start_time_10mins AS end_time,
    DATETIME_DIFF(b.sleep_start_time_10mins,
      b.sleep_start_time,
      MINUTE) AS duration,
    'bedtime_start' AS sleep_event
  FROM
    bpm_stream a
  LEFT JOIN
    sleep_start_end b
  ON
    a.pim_id = b.pim_id
  WHERE
    a.datetime_local BETWEEN b.sleep_start_time
    AND b.sleep_start_time_10mins)
UNION ALL (
  SELECT
    DISTINCT PARSE_DATE("%F",
      '2021-05-04') AS ds,
    a.pim_id,
    a.datetime_local,
    a.bpm,
    b.sleep_end_time AS start_time,
    b.sleep_end_time_10mins AS end_time,
    DATETIME_DIFF(b.sleep_end_time_10mins,
      b.sleep_end_time,
      MINUTE) AS duration,
    'bedtime_end' AS sleep_event
  FROM
    bpm_stream a
  LEFT JOIN
    sleep_start_end b
  ON
    a.pim_id = b.pim_id
  WHERE
    a.datetime_local BETWEEN b.sleep_end_time
    AND b.sleep_end_time_10mins)
"""

In [9]:
test_dta=pd.read_gbq(sql_script,\
                             'research-01-217611', dialect='standard')

In [10]:
test_dta.head(5)

,ds,pim_id,datetime_local,bpm,start_time,end_time,duration,sleep_event
0,2021-05-04,1257990,2021-05-03 12:32:00,117,2021-05-03 12:28:30,2021-05-03 12:38:30,10,bedtime_end
1,2021-05-04,1342089,2021-05-03 05:46:00,107,2021-05-03 05:40:00,2021-05-03 05:50:00,10,bedtime_end
2,2021-05-04,15877,2021-05-03 05:58:00,100,2021-05-03 05:56:00,2021-05-03 06:06:00,10,bedtime_end
3,2021-05-04,1284279,2021-05-03 06:53:00,101,2021-05-03 06:43:00,2021-05-03 06:53:00,10,bedtime_end
4,2021-05-04,1278586,2021-05-03 07:03:00,102,2021-05-03 06:53:30,2021-05-03 07:03:30,10,bedtime_end


In [11]:
test_dta=preprocess(test_dta)

In [13]:
test_dta.head(3)

,ds,pim_id,datetime_local,bpm,start_time,end_time,duration,sleep_event
datetime_local,,,,,,,,
2021-05-03,2021-05-04,63406,2021-05-03,60,2021-05-02 23:31:30,2021-05-03 00:07:29,36,light
2021-05-03,2021-05-04,28890,2021-05-03,61,2021-05-02 23:50:30,2021-05-03 00:00:30,10,bedtime_start
2021-05-03,2021-05-04,136375,2021-05-03,76,2021-05-02 23:50:00,2021-05-03 00:00:00,10,bedtime_start


### Event feature generation

In [14]:
bpm_metrics = [ 
        bpm_features.mean_bpm,
        bpm_features.sd_bpm,
        bpm_features.min_bpm,
        bpm_features.max_bpm,
        bpm_features.count_
    ]

In [15]:
stage_df = pd.DataFrame(
        test_dta['pim_id'].unique(),
        columns = ["pim_id"]
    )

In [16]:
stage_df.head(3)

,pim_id
0,63406
1,28890
2,136375


In [17]:
test_dta[test_dta.sleep_event == 'rem'].head()

,ds,pim_id,datetime_local,bpm,start_time,end_time,duration,sleep_event
datetime_local,,,,,,,,
2021-05-03 00:38:00,2021-05-04,16751934,2021-05-03 00:38:00,80,2021-05-03 00:37:30,2021-05-03 01:02:29,25,rem
2021-05-03 00:39:00,2021-05-04,16751934,2021-05-03 00:39:00,84,2021-05-03 00:37:30,2021-05-03 01:02:29,25,rem
2021-05-03 00:40:00,2021-05-04,63406,2021-05-03 00:40:00,64,2021-05-03 00:40:00,2021-05-03 00:53:29,13,rem
2021-05-03 00:40:00,2021-05-04,166353,2021-05-03 00:40:00,55,2021-05-03 00:39:30,2021-05-03 01:31:59,52,rem
2021-05-03 00:40:00,2021-05-04,16751934,2021-05-03 00:40:00,83,2021-05-03 00:37:30,2021-05-03 01:02:29,25,rem


In [20]:
stage_bpm = aggregate_table(
            test_dta[test_dta.sleep_event == 'rem'],
            ["bpm","pim_id","start_time","end_time","duration","sleep_event"],
            ["pim_id","start_time","end_time","duration","sleep_event"],
            bpm_metrics
        )

In [22]:
stage_bpm

mean_bpm  \
pim_id start_time          end_time            duration sleep_event              
10879  2021-05-03 03:53:00 2021-05-03 04:01:29 8        rem          79.777778   
       2021-05-03 04:12:00 2021-05-03 04:17:29 5        rem          76.500000   
       2021-05-03 05:35:30 2021-05-03 05:46:59 11       rem          73.181818   
       2021-05-03 05:52:30 2021-05-03 06:12:29 20       rem          70.900000   
11733  2021-05-03 01:19:00 2021-05-03 01:44:59 25       rem          72.192308   
...                                                                        ...   
8274   2021-05-03 06:32:00 2021-05-03 06:56:59 24       rem          67.600000   
93916  2021-05-03 00:53:00 2021-05-03 00:58:59 5        rem          71.833333   
       2021-05-03 01:13:30 2021-05-03 01:37:29 24       rem          79.666667   
       2021-05-03 03:54:30 2021-05-03 04:28:59 34       rem          76.970588   
       2021-05-03 05:54:30 2021-05-03 06:17:29 23       rem          76.782609   

                                                                       sd_bpm  \
pim_id start_time          end_time            duration sleep_event             
10879  2021-05-03 03:53:00 2021-05-03 04:01:29 8        rem          1.314684   
       2021-05-03 04:12:00 2021-05-03 04:17:29 5        rem          2.362908   
       2021-05-03 05:35:30 2021-05-03 05:46:59 11       rem          0.935966   
       2021-05-03 05:52:30 2021-05-03 06:12:29 20       rem          1.300000   
11733  2021-05-03 01:19:00 2021-05-03 01:44:59 25       rem          2.602059   
...                                                                       ...   
8274   2021-05-03 06:32:00 2021-05-03 06:56:59 24       rem          2.742262   
93916  2021-05-03 00:53:00 2021-05-03 00:58:59 5        rem          1.572330   
       2021-05-03 01:13:30 2021-05-03 01:37:29 24       rem          3.236081   
       2021-05-03 03:54:30 2021-05-03 04:28:59 34       rem          2.467260   
       2021-05-03 05:54:30 2021-05-03 06:17:29 23       rem          2.244927   

                                                                     min_bpm  \
pim_id start_time          end_time            duration sleep_event            
10879  2021-05-03 03:53:00 2021-05-03 04:01:29 8        rem               77   
       2021-05-03 04:12:00 2021-05-03 04:17:29 5        rem               74   
       2021-05-03 05:35:30 2021-05-03 05:46:59 11       rem               72   
       2021-05-03 05:52:30 2021-05-03 06:12:29 20       rem               67   
11733  2021-05-03 01:19:00 2021-05-03 01:44:59 25       rem               66   
...                                                                      ...   
8274   2021-05-03 06:32:00 2021-05-03 06:56:59 24       rem               62   
93916  2021-05-03 00:53:00 2021-05-03 00:58:59 5        rem               69   
       2021-05-03 01:13:30 2021-05-03 01:37:29 24       rem               72   
       2021-05-03 03:54:30 2021-05-03 04:28:59 34       rem               73   
       2021-05-03 05:54:30 2021-05-03 06:17:29 23       rem               73   

                                                                     max_bpm  \
pim_id start_time          end_time            duration sleep_event            
10879  2021-05-03 03:53:00 2021-05-03 04:01:29 8        rem               82   
       2021-05-03 04:12:00 2021-05-03 04:17:29 5        rem               81   
       2021-05-03 05:35:30 2021-05-03 05:46:59 11       rem               75   
       2021-05-03 05:52:30 2021-05-03 06:12:29 20       rem               73   
11733  2021-05-03 01:19:00 2021-05-03 01:44:59 25       rem               78   
...                                                                      ...   
8274   2021-05-03 06:32:00 2021-05-03 06:56:59 24       rem               73   
93916  2021-05-03 00:53:00 2021-05-03 00:58:59 5        rem               74   
       2021-05-03 01:13:30 2021-05-03 01:37:29 24       rem               86   
       2021-05-03 03:54:30 2

In [28]:
stage_bpm.reset_index().replace({np.nan: None})

,pim_id,start_time,end_time,duration,sleep_event,mean_bpm,sd_bpm,min_bpm,max_bpm,count_
0,10879,2021-05-03 03:53:00,2021-05-03 04:01:29,8,rem,79.777778,1.314684,77,82,9
1,10879,2021-05-03 04:12:00,2021-05-03 04:17:29,5,rem,76.500000,2.362908,74,81,6
2,10879,2021-05-03 05:35:30,2021-05-03 05:46:59,11,rem,73.181818,0.935966,72,75,11
3,10879,2021-05-03 05:52:30,2021-05-03 06:12:29,20,rem,70.900000,1.300000,67,73,20
4,11733,2021-05-03 01:19:00,2021-05-03 01:44:59,25,rem,72.192308,2.602059,66,78,26
...,...,...,...,...,...,...,...,...,...,...
194,8274,2021-05-03 06:32:00,2021-05-03 06:56:59,24,rem,67.600000,2.742262,62,73,25
195,93916,2021-05-03 00:53:00,2021-05-03 00:58:59,5,rem,71.833333,1.572330,69,74,6
196,93916,2021-05-03 01:13:30,2021-05-03 01:37:29,24,rem,79.666667,3.236081,72,86,24
197,93916,2021-05-03 03:54:30,2021-05-03 04:28:59,34,rem,76.970588,2.467260,73,81,34


In [39]:
def calc_event_metrics(period_sleep):
    '''
    Calculate the event bpm values 
    period_sleep : dataframe containing the sleep stages bpm values
    '''
    bpm_metrics = [ 
        bpm_features.mean_bpm,
        bpm_features.sd_bpm,
        bpm_features.min_bpm,
        bpm_features.max_bpm,
        bpm_features.count_
    ]
    
    stage_df=pd.DataFrame()
    
    for key in ['wake', 'light', 'deep', 'rem','awake','asleep','restless','bedtime_start','bedtime_end']:
        stage_bpm = aggregate_table(period_sleep[period_sleep.sleep_event == key],
            ["bpm","pim_id","start_time","end_time","duration","sleep_event"],
            ["pim_id","start_time","end_time","duration","sleep_event"],
            bpm_metrics
        ).reset_index()
        
        stage_df = stage_df.append(stage_bpm,ignore_index=True)

    stage_df = stage_df.replace({np.nan: None})
        
    return stage_df

In [40]:
test_final=calc_event_metrics(test_dta)

In [41]:
test_final.sleep_event.value_counts()

bedtime_end      2716
bedtime_start    1329
light             482
wake              207
rem               199
deep              152
restless           54
asleep             53
awake              11
Name: sleep_event, dtype: int64

In [45]:
test_final[test_final.sleep_event=='deep'].sort_values(by=['pim_id','start_time'])

,pim_id,start_time,end_time,duration,sleep_event,mean_bpm,sd_bpm,min_bpm,max_bpm,count_
689,10879,2021-05-03 03:13:30,2021-05-03 03:43:59,30,deep,78.419355,0.833732,77,80,31
690,10879,2021-05-03 04:40:30,2021-05-03 04:47:59,7,deep,76.714286,0.451754,76,77,7
691,11733,2021-05-03 00:12:30,2021-05-03 01:16:29,64,deep,70.500000,1.075291,69,74,64
692,11733,2021-05-03 02:26:00,2021-05-03 02:30:59,4,deep,69.800000,0.400000,69,70,5
693,137776,2021-05-02 23:44:00,2021-05-03 00:45:29,61,deep,70.760870,1.087174,69,73,46
...,...,...,...,...,...,...,...,...,...,...
836,66108,2021-05-04 00:43:00,2021-05-04 01:04:59,21,deep,71.818182,0.574960,71,73,22
837,66108,2021-05-04 04:14:30,2021-05-04 04:34:59,20,deep,59.300000,2.076054,58,68,20
838,8274,2021-05-03 06:07:30,2021-05-03 06:27:29,20,deep,62.500000,4.043513,60,73,20
839,93916,2021-05-03 03:19:00,2021-05-03 03:43:59,24,deep,70.120000,0.765245,69,72,25


In [48]:
test_final.sort_values(by=['pim_id','start_time'])

,pim_id,start_time,end_time,duration,sleep_event,mean_bpm,sd_bpm,min_bpm,max_bpm,count_
1158,100025,2021-05-02 23:56:00,2021-05-03 00:06:00,10,bedtime_start,56.000000,1.095445,55,58,5
2487,100025,2021-05-03 05:16:00,2021-05-03 05:26:00,10,bedtime_end,57.090909,3.146558,53,62,11
2488,100031,2021-05-03 08:21:00,2021-05-03 08:31:00,10,bedtime_end,62.727273,2.299838,59,67,11
1159,100108,2021-05-03 01:24:00,2021-05-03 01:34:00,10,bedtime_start,63.272727,0.962091,61,64,11
2489,100108,2021-05-03 07:21:00,2021-05-03 07:31:00,10,bedtime_end,85.500000,22.738733,53,112,10
...,...,...,...,...,...,...,...,...,...,...
2486,99450,2021-05-03 12:40:00,2021-05-03 12:50:00,10,bedtime_start,69.636364,3.444963,65,77,11
5199,99450,2021-05-03 16:42:00,2021-05-03 16:52:00,10,bedtime_end,78.000000,3.592922,72,84,11
5200,99654,2021-05-03 06:18:30,2021-05-03 06:28:30,10,bedtime_end,91.363636,6.623275,76,99,11
5201,99680,2021-05-03 04:33:30,2021-05-03 04:43:30,10,bedtime_end,79.700000,3.100000,75,86,10


Wait for meal table ready.